# Discrimination Latin Square Sequences

In [1]:
import scipy.io
import os 
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy 
import scipy.stats as stats
from scipy.stats import sem 
import json
import random
import copy

In [7]:
# load latin square from MATLAB randls function

p = '/Users/pmahableshwarkar/Documents/Depth_Project/latin-square-counterbalancing/randls78.mat'

randls78 = scipy.io.loadmat(p)['ans']

randls78.shape, randls78[0:3]

((78, 78),
 array([[69, 16, 59, 19, 67, 26, 60,  5, 17, 13, 12,  6, 61, 51, 52, 21,
         43, 58, 14, 34, 70, 35, 78,  3,  7, 28, 32,  1, 75, 18, 72,  4,
         25, 56, 37, 23, 64, 20, 73, 39, 46, 54, 45, 76, 55, 47, 63, 38,
         77, 62, 48, 44, 66,  2, 36, 27, 31, 40, 57, 41, 29, 10, 53, 15,
         50, 49, 11, 30, 68, 74, 24,  9,  8, 71, 33, 42, 65, 22],
        [ 5, 26,  7, 49, 24, 18, 15, 13, 14, 25, 39, 28, 35, 67, 71, 36,
         21,  8, 69, 32,  1, 63,  4, 29, 55, 34, 40, 33, 58, 62, 70, 72,
         51, 77, 78, 66, 17,  9, 22, 46, 45, 20, 37, 48, 10, 59, 53, 74,
         31, 47, 61, 57, 54, 50, 16, 23, 30, 38, 60,  2, 11, 56, 43, 19,
         44, 68, 73, 27, 65, 41, 12,  3, 75,  6, 64, 76, 52, 42],
        [32, 40, 71, 42, 35,  1, 30, 14, 57,  9, 46, 73,  4, 10, 22, 66,
         13, 16, 47, 15, 51, 23, 43, 67, 64,  7, 69, 38, 19, 29, 41, 33,
          6, 37, 39, 36, 55, 12, 54, 25, 58, 18,  3, 77, 74, 70, 76, 60,
         65, 75,  5,  8, 61, 53, 45, 68, 48, 11, 56, 1

In [9]:
# load an existing balanced VE sequence and seperate into 4 blocks
discrim_seq_path = '/Users/pmahableshwarkar/Documents/Depth_Project/latin-square-counterbalancing/adjusted_j13.json'

with open(discrim_seq_path, 'r') as f:
    discrim_seq = json.load(f)
    


In [10]:
to_remove = ['000505_2014-06-08_23-03-23_260595134347_rgbf000091-resize_0',
 '002579_2014-06-24_14-08-03_094959634447_rgbf000061-resize_2',
 '003546_2014-05-22_15-48-53_094959634447_rgbf004816-resize_3',
 '000830_2014-06-04_19-49-31_260595134347_rgbf000096-resize_2',
 '002460_2014-06-28_20-35-43_260595134347_rgbf000028-resize_4',
 '002468_2014-06-28_20-38-39_260595134347_rgbf000045-resize_1',
 '002139_2014-06-25_21-19-37_260595134347_rgbf000124-resize_2',
 '002365_2014-06-28_20-11-21_260595134347_rgbf000102-resize_1',
 '002700_2014-06-22_11-27-02_094959634447_rgbf000124-resize_3',
 '001724_2014-06-26_19-28-50_260595134347_rgbf000037-resize_7',
 '002464_2014-06-28_20-36-57_260595134347_rgbf000029-resize_0',
 '003288_2014-05-14_21-50-30_094959634447_rgbf000028-resize_3',
 '002305_2014-06-28_19-56-50_260595134347_rgbf000102-resize_7',
 '001181_2014-06-17_15-50-57_260595134347_rgbf000106-resize_4',
 '000555_2014-06-09_22-42-23_260595134347_rgbf000037-resize_4',
 '003092_2014-06-24_21-01-49_094959634447_rgbf000049-resize_2',
 '001532_2014-06-20_16-50-41_260595134347_rgbf000081-resize_4',
 '000636_2014-06-08_16-39-02_260595134347_rgbf000108-resize_4']

In [11]:
# removes trials that have bad images, and removes catch trials 

count_remove = 0
depths = []
cleaned_discrim_seq = []
catch = []
for trial in discrim_seq:
    if trial['sequence'] != 'catch_trial':
        p0 = trial['image_path_target_0'].split('/')[1]
        p1 = trial['image_path_target_1'].split('/')[1]
        if p0 in to_remove:
            count_remove += 1
        else:
            if p1 in to_remove:
                count_remove += 1
                
        if p0 not in to_remove:
            if p1 not in to_remove:
                d0 = trial['depth_0']
                depths.append(d0)
                d1 = trial['depth_1']
                depths.append(d1)
                cleaned_discrim_seq.append(trial)
    else:
        catch.append(trial)

len(cleaned_discrim_seq), len(catch)

(78, 8)

In [12]:
# Discrimination_AccurateResponse_Distribution(cleaned_discrim_seq)

In [13]:
def randls_to_sequence(randls_row, block):
    '''
    Uses indeces from the randls matrix row to build sequence
    
    Args:
        randls_row - one row of the randls matrix
        block - one block out of the four in the VE sequence
    Returns:
        VE trials in sequence
    '''
    sequence = []
    for elem in randls_row:
        if elem == len(randls_row):
            idx = 0
        else:
            idx = elem
        sequence.append(block[idx])
    return sequence 

def main_randls_blocks(randls, block):
    '''
    Converts each randls row into a sequence 
    
    Args:
        randls - randls matrix from MATLAB
        block - one block out of the four in the VE sequence
    Returns:
        All sequences from the randls matrix in trial sequence format
    '''
    block_sequences = []
    for i in range(randls.shape[0]):
        row = randls[i]
        seq = randls_to_sequence(row, block)
        block_sequences.append(seq)
    return block_sequences

In [14]:
full_randls_seqs = main_randls_blocks(randls78, cleaned_discrim_seq)


In [15]:
len(full_randls_seqs), len(full_randls_seqs[0])

(78, 78)

In [16]:
# shuffle the order of the sequences
random.shuffle(full_randls_seqs)


In [193]:
15-6, 23-17, 34-23, 51-34, 60-51, 70-60, 76-70



(9, 6, 11, 17, 9, 10, 6)

In [247]:
# build the blocks together + catch trials
sequences78 = []
# pick 47/94 sequences
random.shuffle(catch)
for i in range(len(full_randls_seqs)):
    fseq = full_randls_seqs[i]
    catch_locs = [6, 15, 23, 34, 51, 60, 70, 76]
    for i in range(len(catch)):
        fseq.insert(catch_locs[i], catch[i])
    
    sequences78.append(fseq)
    
len(sequences78), len(sequences78[0]), sequences78[0][0]

(78,
 86,
 {'sequence': 'j13',
  'duration': 1000,
  'depth_0': 4.2115,
  'depth_1': 3.753,
  'image_path_target_0': 'depth_discrimination_stimuli/001133_2014-06-17_14-49-16_260595134347_rgbf000100-resize_0/001133_2014-06-17_14-49-16_260595134347_rgbf000100-resize_0-target.png',
  'image_path_target_1': 'depth_discrimination_stimuli/001132_2014-06-17_14-48-54_260595134347_rgbf000100-resize_2/001132_2014-06-17_14-48-54_260595134347_rgbf000100-resize_2-target.png',
  'mask_path': 'masks/mask_39.jpg',
  'fixation_path': 'fixation.jpg'})

In [248]:
# set all trial durations to 250 ms
for seq in sequences78:
    for trial in seq:
        trial['duration'] = 250

sequences78[0][0]

{'sequence': 'j13',
 'duration': 250,
 'depth_0': 4.2115,
 'depth_1': 3.753,
 'image_path_target_0': 'depth_discrimination_stimuli/001133_2014-06-17_14-49-16_260595134347_rgbf000100-resize_0/001133_2014-06-17_14-49-16_260595134347_rgbf000100-resize_0-target.png',
 'image_path_target_1': 'depth_discrimination_stimuli/001132_2014-06-17_14-48-54_260595134347_rgbf000100-resize_2/001132_2014-06-17_14-48-54_260595134347_rgbf000100-resize_2-target.png',
 'mask_path': 'masks/mask_39.jpg',
 'fixation_path': 'fixation.jpg'}

In [249]:
# deepcopy of sequences - prevents alterations 
sequences78_250ms = copy.deepcopy(sequences78)

In [250]:
# set all trial durations to 1000 ms
for seq in sequences78:
    for trial in seq:
        trial['duration'] = 1000

sequences78[0][0]

{'sequence': 'j13',
 'duration': 1000,
 'depth_0': 4.2115,
 'depth_1': 3.753,
 'image_path_target_0': 'depth_discrimination_stimuli/001133_2014-06-17_14-49-16_260595134347_rgbf000100-resize_0/001133_2014-06-17_14-49-16_260595134347_rgbf000100-resize_0-target.png',
 'image_path_target_1': 'depth_discrimination_stimuli/001132_2014-06-17_14-48-54_260595134347_rgbf000100-resize_2/001132_2014-06-17_14-48-54_260595134347_rgbf000100-resize_2-target.png',
 'mask_path': 'masks/mask_39.jpg',
 'fixation_path': 'fixation.jpg'}

In [251]:
# deepcopy of sequences - prevents alterations 
sequences78_1000ms = copy.deepcopy(sequences78)

In [252]:
sequences78_250ms[0][0]['duration'], sequences78_1000ms[0][0]['duration']

(250, 1000)

In [253]:
dest_250ms = 'XX'
dest_1000ms = 'XX'

In [254]:
# save sequences as jsons 
num = 0
for seq in sequences78_250ms:
    with open(dest_250ms + 'Discrim250_randls_' + str(num)+ '.json', 'w') as outfile:
        json.dump(seq, outfile)
        num += 1

In [255]:
# save sequences as jsons 
num = 0
for seq in sequences78_1000ms:
    with open(dest_1000ms + 'Discrim1000_randls_' + str(num)+ '.json', 'w') as outfile:
        json.dump(seq, outfile)
        num += 1

# Flip sequences from front to back and within trials 



In [256]:
import os 

def load_sequence(jsonpath):
    return json.load(open(jsonpath))

def flip_image_order(previous_seq):
    flipped = previous_seq
    for i in range(len(previous_seq)):
        og_depth0 = previous_seq[i]['depth_0']
        og_depth1 = previous_seq[i]['depth_1']
        og_path0 = previous_seq[i]['image_path_target_0']
        og_path1 = previous_seq[i]['image_path_target_1']
        
        flipped[i]['depth_0'] = og_depth1
        flipped[i]['depth_1'] = og_depth0
        flipped[i]['image_path_target_0'] = og_path1
        flipped[i]['image_path_target_1'] = og_path0
        
    return flipped

def create_flipped_seqs(jsonpath, exit, name):
    master = load_sequence(jsonpath)
    # reverse trial order
    master.reverse()
    # flip image order witin each trial
    r = flip_image_order(master)
    r_path = exit + '/' + name + '_rotated.json' # duration rotated sequence
    #creates json file for the sequence 
    with open(r_path, 'w') as f:
        json.dump(r , f)

        
def main_seq_flips(json_folderpath, exit):
    """
    Create rotated sequence for each sequence in the folder
    """
    for file in os.listdir(json_folderpath):
        name = file.split(".")[0]
        jsonpath = json_folderpath + "/" + file
        try:
            create_flipped_seqs(jsonpath, exit, name)
        except:
            print("Failed to create json flips for: ", file)

In [257]:
main_seq_flips(dest_250ms,dest_250ms)


Failed to create json flips for:  .DS_Store


In [258]:
main_seq_flips(dest_1000ms,dest_1000ms)


Failed to create json flips for:  .DS_Store


In [259]:
def Discrimination_AccurateResponse_Distribution(jsondata):
    """
    args: json data in a dictionary format
    returns: [count0,count1] 
             where count0 is the number of trials where image 0 is the correct answer 
             where count1 is the number of trials where image 1 is the correct answer
             
    Question: Which target is CLOSER to you? 
    """
    count0 = 0
    count1 = 0
    equal = 0
    
    for trial in jsondata:
        if trial['sequence'] != 'catch_trial':
            depth0 = trial['depth_0']
            depth1 = trial['depth_1']
            if depth0 < depth1:
                count0 += 1
            if depth0 > depth1:
                count1 += 1
            if depth0 == depth1:
                equal += 1
    return [count0, count1, equal]

In [260]:
main_distribution = {}

for js in os.listdir(dest_250ms):
    if '.json' in js:
        j_path = dest_250ms + '/' + js
        with open(j_path) as f:
            data = json.load(f)
            main_distribution[js] = Discrimination_AccurateResponse_Distribution(data)

main_distribution  

{'Discrim250_randls_0.json': [39, 39, 0],
 'Discrim250_randls_0_rotated.json': [39, 39, 0],
 'Discrim250_randls_1.json': [39, 39, 0],
 'Discrim250_randls_10.json': [39, 39, 0],
 'Discrim250_randls_10_rotated.json': [39, 39, 0],
 'Discrim250_randls_11.json': [39, 39, 0],
 'Discrim250_randls_11_rotated.json': [39, 39, 0],
 'Discrim250_randls_12.json': [39, 39, 0],
 'Discrim250_randls_12_rotated.json': [39, 39, 0],
 'Discrim250_randls_13.json': [39, 39, 0],
 'Discrim250_randls_13_rotated.json': [39, 39, 0],
 'Discrim250_randls_14.json': [39, 39, 0],
 'Discrim250_randls_14_rotated.json': [39, 39, 0],
 'Discrim250_randls_15.json': [39, 39, 0],
 'Discrim250_randls_15_rotated.json': [39, 39, 0],
 'Discrim250_randls_16.json': [39, 39, 0],
 'Discrim250_randls_16_rotated.json': [39, 39, 0],
 'Discrim250_randls_17.json': [39, 39, 0],
 'Discrim250_randls_17_rotated.json': [39, 39, 0],
 'Discrim250_randls_18.json': [39, 39, 0],
 'Discrim250_randls_18_rotated.json': [39, 39, 0],
 'Discrim250_randls_

In [261]:
main_distribution = {}

for js in os.listdir(dest_1000ms):
    if '.json' in js:
        j_path = dest_1000ms + '/' + js
        with open(j_path) as f:
            data = json.load(f)
            main_distribution[js] = Discrimination_AccurateResponse_Distribution(data)

main_distribution 

{'Discrim1000_randls_0.json': [39, 39, 0],
 'Discrim1000_randls_0_rotated.json': [39, 39, 0],
 'Discrim1000_randls_1.json': [39, 39, 0],
 'Discrim1000_randls_10.json': [39, 39, 0],
 'Discrim1000_randls_10_rotated.json': [39, 39, 0],
 'Discrim1000_randls_11.json': [39, 39, 0],
 'Discrim1000_randls_11_rotated.json': [39, 39, 0],
 'Discrim1000_randls_12.json': [39, 39, 0],
 'Discrim1000_randls_12_rotated.json': [39, 39, 0],
 'Discrim1000_randls_13.json': [39, 39, 0],
 'Discrim1000_randls_13_rotated.json': [39, 39, 0],
 'Discrim1000_randls_14.json': [39, 39, 0],
 'Discrim1000_randls_14_rotated.json': [39, 39, 0],
 'Discrim1000_randls_15.json': [39, 39, 0],
 'Discrim1000_randls_15_rotated.json': [39, 39, 0],
 'Discrim1000_randls_16.json': [39, 39, 0],
 'Discrim1000_randls_16_rotated.json': [39, 39, 0],
 'Discrim1000_randls_17.json': [39, 39, 0],
 'Discrim1000_randls_17_rotated.json': [39, 39, 0],
 'Discrim1000_randls_18.json': [39, 39, 0],
 'Discrim1000_randls_18_rotated.json': [39, 39, 0],